In [ ]:
#Installing and importing major needs
!pip install openai
!pip install transformers
import json
import numpy as np
import pandas as pd
import tensorflow as tf 
from matplotlib import pyplot as plt
from google.colab import drive
import os
!pip install -q -U tensorflow-text ## install the package for NLP tasks in tensorflow
import tensorflow_hub as hub
import tensorflow_text
import tensorflow as tf
from tensorflow import keras
import pickle

drive.mount('/content/drive')
%matplotlib inline


In [ ]:

'''
users = []
numReviewTotal = 1e6
with open('/content/drive/MyDrive/HoDL Project - Group 7/yelp_dataset/yelp_academic_dataset_review.json') as fl:
    for i, line in enumerate(fl):
        users
      .append(json.loads(line))
        if i+1 >= numReviewTotal:
            break
df = pd.DataFrame(users)
df.head()
fl.close()
pd.DataFrame.to_csv(df,'/content/drive/MyDrive/HoDL Project - Group 7/yelp_dataset/Review_1M.csv')
'''

df = pd.read_csv('/content/drive/MyDrive/HoDL Project - Group 7/yelp_dataset/Review_1M.csv',nrows=50000)


In [ ]:
#Transforming the DF to have only the label and the message & 1/5
df_oficial = df[(df.stars == 1.0) | (df.stars == 5.0)]# | (df.stars == 3.0)]
#df_oficial = df_oficial[['stars', 'text']]

df_oficial['stars'] = pd.Categorical(df_oficial.stars).codes
df_oficial.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5.0,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3.0,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5.0,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4.0,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15


In [ ]:
x=list(df_oficial['text'])
y=list(df_oficial['stars'])
z=list(df_oficial['business_id'])

## Train test split
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test, business_id_train, business_id_test = train_test_split(x,y,z, test_size = 0.20, random_state = 0)

print("The number of lines in x_train is:") 
len(x_train) 



In [ ]:
bert_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

bert_layers = 12
bert_units = 768
bert_heads = 12

bert_encoder = f'https://tfhub.dev/tensorflow/bert_en_uncased_L-{bert_layers}_H-{bert_units}_A-{bert_heads}/4'

In [ ]:
max_length = 512
preprocessor = hub.load(bert_preprocess)
encoder = hub.KerasLayer(bert_encoder, trainable=False)

def bert_textvect(x):
  input = keras.layers.Input(shape=(), dtype=tf.string)
  tokenized_input = hub.KerasLayer(preprocessor.tokenize)(input)
  bert_pack_inputs = hub.KerasLayer(preprocessor.bert_pack_inputs, arguments=dict(seq_length=max_length))
  output = bert_pack_inputs([tokenized_input])
  model = keras.Model(input, output)
  result = model.predict(x)
  return result


def bert_features(x):
  inputs = dict(
    input_word_ids=keras.layers.Input(shape=(max_length,), dtype=tf.int32),
    input_mask=keras.layers.Input(shape=(max_length,), dtype=tf.int32),
    input_type_ids=keras.layers.Input(shape=(max_length,), dtype=tf.int32),
  )

  output = encoder(inputs)['sequence_output'][:, 0, :]
  model = keras.Model(inputs, output)
  return model.predict(x)


In [ ]:
X_bert_train = bert_textvect(x_train)
X_bert_test = bert_textvect(x_test)

features_test = bert_features(X_bert_test)
features_train = bert_features(X_bert_train)

(1000, 512)

In [ ]:
with open('/content/drive/MyDrive/HoDL Project - Group 7/yelp_dataset/1_5Star_Reviews.pkl', 'wb') as f:
  pickle.dump({'X_bert_train':X_bert_train,'X_bert_test':X_bert_test,'x_train': x_train, 'x_test': x_test, 'features_test':features_test,'features_train':features_train,'y_train': y_train, 'y_test':y_test},f)


(1000, 512)